# 第4回課題：夏目漱石の「吾輩は猫である」っぽい文を生成してみよう！

**課題は結果が出力されている状態で保存して提出してください！**

## 課題１：分かち書き文の生成

`text/wagahaiwa_nekodearu_org.txt`には、夏目漱石の小説「吾輩は猫である」の本文が記録されています。   
このテキストを半角スペース区切りの分かち書き文に変換して、`text/wagahaiwa_nekodearu_wakati.txt`に保存してください。  
なお、改行は削除せずにそのまま残してしておいてください。

In [27]:
test_data = open("text/wagahaiwa_nekodearu_org.txt", "r")
test_data2 = open("text/wagahaiwa_nekodearu_wakati.txt", "w")

contents = test_data.read()


sentences = contents.split()

from janome.tokenizer import Tokenizer
wakatis = []
t = Tokenizer() # <== これは辞書を読み込み、解析プログラムを起動状態にします。メモリを食うので、１つのプログラムで１回だけ実行して、あとは使いまわしてください。
for sentence in sentences:
    tokens = t.tokenize(sentence, wakati=True)
    wakati= ''
    for token in tokens:
        wakati += token + ' '
    wakati = wakati.rstrip()
    test_data2.write(wakati)
    wakatis.append(wakati)

test_data.close()
test_data2.close()

['吾輩 は 猫 で ある 。 名前 は まだ 無い 。']


冒頭の1文を出力してください。  
次のような文が出てきたら正解です。  
```吾輩 は 猫 で ある 。 名前 は まだ 無い 。 ```

In [31]:
print(wakatis[:1])

['吾輩 は 猫 で ある 。 名前 は まだ 無い 。']


## 課題２：ランダム文生成

下のセルで、課題１で作成した`text/wagahaiwa_nekodearu_wakati.txt`を入力として、統計的tri-gramモデルを学習してください。  
学習モデルの変数名は`lm`とします。   

In [141]:
from nltk.lm import Vocabulary
from nltk.lm.models import MLE
from nltk.util import ngrams

file = 'text/wagahaiwa_nekodearu_wakati.txt'

words = [('<BOP> ' + l + ' <EOP>').split() for l in open(file, 'r', encoding='utf-8').readlines()]


# 読み込んだ小説集の語彙（異なり単語）を収集
# Vocabularyは1次元のリストを受け取るが、wordsは2次元のリストなので、
# wordsを内包表記で2次元から1次元に変換してからVocabularyに渡しています
vocab = Vocabulary([item for sublist in words for item in sublist])

# 語彙の一覧を表示させたいなら下4行のコメントを有効にする
'''num = 0
for v in sorted(vocab.counts):
    print('{:d}\t{:s}'.format(num,v))
    num += 1
'''
#print('Vocabulary size: ' + str(len(vocab))) # 語彙サイズ（単語の種類数）

text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを生成

n = 3
lm = MLE(order = n, vocabulary = vocab) # 最尤推定法（Maximum Likelihood Estimation)による統計的n-gram言語モデルの準備
lm.fit(text_trigrams) # 上で生成したtri-gramを使って言語モデルを学習

学習したモデル`lm`を使って、以下のセルで「吾輩は」から始まる文を10文生成してください。   
生成文は「吾輩は」から始めて、`。`あるいは`」`で終わる一文を基本としますが、より自然な文となるよう、独自の改良を加えても構いません。   
**生成文が出力されている状態で保存して提出してください。**

In [140]:
### ランダム文生成 ####
# contextから始まる文を生成

# 最初の2単語はいろいろと変えてみましょう

for i in range(0, 10):
    context = ['吾輩', 'は']
    #context = ['水', '生卵']
    # contextのうち最後の2単語から次に繋がる確率0じゃない単語をランダムに選ぶ
    while True:
        w = lm.generate(text_seed=context)
        context.append(w) # 選ばれた単語をcontextに連結 
        #if w == '。' or w == '」「'or w == '」'or w=='。「':#ここ下の行と選択'。'だけのほうが自然になる？
        if w == '。':
            sentence = ''
            for word in context:
                sentence += word
            if w == '」「' or w=='。「':
                sentence = sentence.rstrip('「')
            print(sentence)
            break

吾輩は無論下た向きでたくさんだと振りむく途端に自己が恃みになりました。
吾輩は突然鈴木君はやはり音楽の出来るものです。
吾輩は心中ひそかに慶賀の意を表しない。「いや、御金を出して刈り込ませて、しばらく考え込んでいる。
吾輩はいろいろあるが、主人はたちまち大きな声を聞いて見るとまるで剣もほろろの挨拶をした心持ち好く日の当る所に吶喊しているくらいだ。
吾輩は死ぬまで焦ったって片付く事がないから不平だ。
吾輩はこれは昨日火事で焚き出されたのだ。
吾輩は日本の猫でも何もしない男である。
吾輩は今でも残っていたいの勇猛心の凝って骨となり得るの途に窮するかも知れない。
吾輩は金田夫婦と見える。
吾輩は彼と近付になって昏睡仮死の庸人と相互する以上はいかなる種類の運動がしいものはいかなる事があって、両眼からかけ始めた。


たとえば次のような文が生成されます（これは一例です）。
```
吾輩は文明の咒詛だ。
吾輩はにゃあにゃあと甘えるごとく、雲を行くがごとく腹内に誘致する職務を帯びている。
吾輩はちょっとむっとする。
吾輩は無論しやせずして来た。
吾輩はやはり地面の上を摩擦した女で、気の利かぬ主人は一人が「へえアンドレア・デル・サルトが言ったがそれぎれで手掛りがないから、おとなしく独乙皇帝陛下のように述べ立てる。
吾輩は謹んで拝見すると云わぬばかりの顔も遠慮なく見るし、Ｔ１Ｔ2……。
吾輩は少からざる尊敬をもって反覆読誦したのね。
吾輩は年来美学上の利害もないんです。
吾輩はこの近辺でやられて、近火の手でさあ。
吾輩は波を打った。
```